<a href="https://colab.research.google.com/github/lawesworks/Genesis/blob/main/yolo8_on_fire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
MY_SECRET_KEY=""

In [ ]:
!pip install -q roboflow ultralytics

In [ ]:
import os
os.environ["ROBOFLOW_API_KEY"] = MY_SECRET_KEY

In [ ]:
from roboflow import Roboflow
import os

rf = Roboflow(api_key=os.environ["ROBOFLOW_API_KEY"])
project = rf.workspace("-jwzpw").project("continuous_fire")
dataset = project.version(1).download("yolov8")  # adjust version if needed

In [ ]:
import glob, os

yaml_files = glob.glob("/content/**/data.yaml", recursive=True)
yaml_files

['/content/continuous_fire-1/data.yaml']

In [ ]:
from ultralytics import YOLO

DATA_YAML_PATH = "/content/continuous_fire-1/data.yaml"

model = YOLO("yolov8n.pt")  # small + fast starter model
results = model.train(
    data=DATA_YAML_PATH,
    epochs=10,
    imgsz=640,
    batch=16
)


In [ ]:
import glob
glob.glob("runs/detect/train*/weights/best.pt")

In [ ]:
best_model_path = glob.glob("runs/detect/train3/weights/best.pt")[-1]
model = YOLO(best_model_path)

model.predict(source="continuous_fire-1/test/images", save=True, conf=0.25)


In [ ]:
model.predict(source="/content/fire.jpg", save=True, conf=0.25)

In [ ]:
# Only run if not already installed
from IPython.display import Image
Image(filename=f'/content/runs/detect/train3/results.png', width=600)

In [ ]:
Image(filename=f'/content/runs/detect/train3/confusion_matrix.png', width=600)

In [ ]:
model.val(data=DATA_YAML_PATH)